In [1]:
pwd

'/home/wsuser/work'

In [2]:
!pip install keras==2.2.4
!pip install tensorflow 

     |████████████████████████████████| 312 kB 24.1 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 19.9 MB/s eta 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.2 requires keras<2.8,>=2.7.0, but you have keras 2.2.4 which is incompatible.
     |████████████████████████████████| 1.3 MB 20.6 MB/s eta 0:00:01
  Attempting uninstall: keras
    Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


In [3]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [4]:
imageSize = [224, 224]

trainPath = r"/content/drive/MyDrive/detection/Dataset/train"

testPath = r"/content/drive/MyDrive/detection/Dataset/test"


In [5]:
# adding preprocessing layers to the front of vgg

vgg = VGG16(input_shape=imageSize + [3], weights='imagenet',include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


In [6]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [7]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)

In [8]:
prediction = Dense(17, activation='softmax')(x)

In [9]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [10]:
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'], run_eagerly=True
)


In [12]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [13]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='zHzhp2P6AiL7y2umik_7K-FlqhpB_RFcAFpsXZtzXCfF',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'naildiagnosis-donotdelete-pr-ncxeoi3a6fvj2j'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [14]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [15]:
pwd

'/home/wsuser/work'

In [16]:
import os
filenames = os.listdir('/home/wsuser/work/Dataset/train')

In [17]:
training_set = train_datagen.flow_from_directory("/home/wsuser/work/Dataset/train",
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory("/home/wsuser/work/Dataset/test",
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 655 images belonging to 17 classes.
Found 183 images belonging to 17 classes.


In [18]:
training_set.class_indices

{'Darier_s disease': 0,
 'Muehrck-e_s lines': 1,
 'aloperia areata': 2,
 'beau_s lines': 3,
 'bluish nail': 4,
 'clubbing': 5,
 'eczema': 6,
 'half and half nailes (Lindsay_s nails)': 7,
 'koilonychia': 8,
 'leukonychia': 9,
 'onycholycis': 10,
 'pale nail': 11,
 'red lunula': 12,
 'splinter hemmorrage': 13,
 'terry_s nail': 14,
 'white nail': 15,
 'yellow nails': 16}

In [19]:
len(training_set)

21

In [20]:
import sys
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set)//3,
  validation_steps=len(test_set)//3)

/tmp/wsuser/ipykernel_209/405012155.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/5
7/7 [==============================] - 278s 40s/step - loss: 3.9433 - accuracy: 0.0402 - val_loss: 3.2703 - val_accuracy: 0.0938
Epoch 2/5
7/7 [==============================] - 275s 40s/step - loss: 3.0983 - accuracy: 0.1429 - val_loss: 2.7074 - val_accuracy: 0.1875
Epoch 3/5
7/7 [==============================] - 274s 39s/step - loss: 2.9247 - accuracy: 0.1696 - val_loss: 2.9936 - val_accuracy: 0.1719
Epoch 4/5
7/7 [==============================] - 275s 40s/step - loss: 2.8475 - accuracy: 0.1920 - val_loss: 2.5412 - val_accuracy: 0.2656
Epoch 5/5
7/7 [==============================] - 255s 36s/step - loss: 2.2929 - accuracy: 0.3092 - val_loss: 2.2174 - val_accuracy: 0.3594


In [21]:
#save the model
model.save("vgg-16-nail-disease.h5")

In [22]:
!tar -zcvf Naildiagnosis-model_new.tgz vgg-16-nail-disease.h5 

vgg-16-nail-disease.h5


In [23]:
ls -1

Dataset/
Naildiagnosis-model_new.tgz
vgg-16-nail-disease.h5


In [24]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 19.9 MB/s eta 0:00:01



Test The Model

In [25]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                    "apikey":"lwEJttJa9ML1OHfiL0Ah3KhVjttm7MaV8WUryPkTjzzN"
                  }
client = APIClient(wml_credentials)

In [26]:
client = APIClient(wml_credentials)

In [27]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [28]:
space_uid = guid_from_space_name(client,'Diagnosis')
print("Space UID = " + space_uid)

Space UID = 7aa6b327-b62f-4c1e-931a-9daf3cc21c5c


In [29]:
client.set.default_space(space_uid)

'SUCCESS'

In [30]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [31]:
software_space_uid = client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [32]:
model_details = client.repository.store_model(model='Naildiagnosis-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:'tensorflow_2.7',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})
                                             
    


In [33]:
model_id = client.repository.get_model_id(model_details)

In [34]:
model_id

'e11dde86-06db-4d8f-9160-4b6592a50723'

In [35]:
client.repository.download(model_id,'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'

In [40]:
#load saved model file
model=load_model('vgg-16-nail-disease.h5')

In [51]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='zHzhp2P6AiL7y2umik_7K-FlqhpB_RFcAFpsXZtzXCfF',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'naildiagnosis-donotdelete-pr-ncxeoi3a6fvj2j'
object_key = '1.PNG'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
img = image.load_img('my Drive',target_size = (224,224))


SyntaxError: invalid syntax (1062337842.py, line 26)

In [ ]:
#convert image to array format
x=image.img_to_array(img)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x.shape

In [ ]:
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

In [ ]:
model.predict(img_data)

In [ ]:
output=np.argmax(model.predict(img_data), axis=1)


In [ ]:
index=['Darier_s disease', 'Muehrck-e_s lines', 'aloperia areata', 'beau_s lines', 'bluish nail',
       'clubbing','eczema','half and half nailes (Lindsay_s nails)','koilonychia','leukonychia',
       'onycholycis','pale nail','red lunula','splinter hemmorrage','terry_s nail','white nail','yellow nails']
result = str(index[output[0]])
result